In [1]:
import gradio as gr
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from PIL import Image
import json
import os
import numpy as np
from util import *

face_detector = pipeline(Tasks.face_detection, model='gaosheng/face_detect')
# face_recognizer = pipeline(Tasks.face_recognition, model='damo/cv_ir101_facerecognition_cfglint')
face_recognizer = pipeline(Tasks.face_recognition, model='iic/cv_ir101_facerecognition_cfglint')
face_bank = load_face_bank('face_bank/', face_recognizer)

def inference(img: Image, draw_detect_enabled, detect_threshold, sim_threshold) -> json:
    img = resize_img(img)
    img = img.convert('RGB')
    detection_result = face_detector(img)

    boxes = np.array(detection_result[OutputKeys.BOXES])
    scores = np.array(detection_result[OutputKeys.SCORES])
    faces = []

    for i in range(len(boxes)):
        score = scores[i]
        if score < detect_threshold:
            continue
        box = boxes[i]
        face_embedding = get_face_embedding(img, box, face_recognizer)
        name, sim = get_name_sim(face_embedding, face_bank)
        if name is None:
            continue
        if sim < sim_threshold:
            faces.append({'box': box, 'name': '未知', 'sim': sim})
        else:
            faces.append({'box': box, 'name': name, 'sim': sim})
    rows = get_rows(faces)
    row_names = get_row_names(faces, rows)
    draw_name(img, row_names)
    if draw_detect_enabled:
        draw_faces(img, faces)
    return img, get_row_names_text(row_names)

examples = ['example.jpg']

with gr.Blocks() as demo:
    with gr.Row():
        draw_detect_enabled = gr.Checkbox(label="是否画框", value=True)
        detect_threshold = gr.Slider(label="检测阈值", minimum=0, maximum=1, value=0.3)
        sim_threshold = gr.Slider(label="识别阈值", minimum=0, maximum=1, value=0.3)
    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", height=350)
            submit = gr.Button("提交")
        with gr.Column():
            img_output = gr.Image(type="pil")
            name_output = gr.Text(label="人名")
    submit.click(
        fn=inference, 
        inputs=[img_input, draw_detect_enabled, detect_threshold, sim_threshold],
        outputs=[img_output, name_output])
    gr.Examples(examples, inputs=[img_input])

demo.launch(share=True)

2024-12-08 11:49:00.650504: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 11:49:00.694335: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-08 11:49:01.646119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-12-08 11:49:04,061 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
2024-12-08 11:49:04,062 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect.
2024-12-08 11:49:04,066 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
/usr/local/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info The read operation timed out
  data = fetch_version_info()
/usr/local/lib/py

load checkpoint from local path: /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect/pytorch_model.pt


2024-12-08 11:49:16,045 - modelscope - INFO - load model done


2024-12-08 11:49:16,925 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2024-12-08 11:49:17,286 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint
2024-12-08 11:49:17,287 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint.
2024-12-08 11:49:17,298 - modelscope - WARNING - No preprocessor field found in cfg.
2024-12-08 11:49:17,298 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-12-08 11:49:17,299 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint'}. trying to build by task and model information.
2024-12-08 11:49:17,299 - modelscope - WARNING - Find task: face-recognition, model type: None. Insufficient information 

2024-12-08 11:49:17,981 - modelscope - WARNING - Model revision not specified, use revision: v1.1
2024-12-08 11:49:18,315 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd
2024-12-08 11:49:18,316 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd.
2024-12-08 11:49:18,321 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" inst

load checkpoint from local path: /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd/pytorch_model.pt


2024-12-08 11:49:19,945 - modelscope - INFO - face recognition model loaded!


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c0303192c16ca6ad28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:333: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:369: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  warnings.warn(
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet